In [1]:
import urllib.request, xml.etree.ElementTree as ET

url = ("https://datenservice.tradinghub.eu/XmlInterface/getXML.ashx"
       "?ReportId=PricesEnergyImbalance&Start=01-10-2021&End=31-10-2021")
req = urllib.request.Request(url, headers={"User-Agent": "YourOrg-YourApp/1.0"})
with urllib.request.urlopen(req, timeout=60) as r:
    xml = r.read()

root = ET.fromstring(xml)
# inspect the first few elements
for child in list(root)[:5]:
    print(child.tag, {k:v for k,v in child.attrib.items()})

{http://www.w3.org/2001/XMLSchema}schema {'targetNamespace': 'urn:schemas-microsoft-com:sql:SqlRowSet1', 'elementFormDefault': 'qualified'}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesEnergyImbalance {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesEnergyImbalance {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesEnergyImbalance {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesEnergyImbalance {}


In [2]:
import urllib.request, xml.etree.ElementTree as ET

url = ("https://datenservice.tradinghub.eu/XmlInterface/getXML.ashx"
       "?ReportId=PricesFlexibilityChargeIntraday")
req = urllib.request.Request(url, headers={"User-Agent": "YourOrg-YourApp/1.0"})
with urllib.request.urlopen(req, timeout=60) as r:
    xml = r.read()

root = ET.fromstring(xml)
# Explore what fields THE returns today
for row in root:
    print(row.tag, row.attrib)

{http://www.w3.org/2001/XMLSchema}schema {'targetNamespace': 'urn:schemas-microsoft-com:sql:SqlRowSet1', 'elementFormDefault': 'qualified'}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-microsoft-com:sql:SqlRowSet1}PricesFlexibilityChargeIntraday {}
{urn:schemas-

In [4]:
import urllib.request, xml.etree.ElementTree as ET
import csv, sys

URL = "https://datenservice.tradinghub.eu/XmlInterface/getXML.ashx?ReportId=PricesFlexibilityChargeIntraday"
UA = "YourOrg-YourApp/1.0 (you@example.com)"  # use a real UA per THE's rules

def strip_ns(tag):
    return tag.split('}', 1)[-1] if '}' in tag else tag

def fetch_xml(url):
    req = urllib.request.Request(url, headers={"User-Agent": UA})
    with urllib.request.urlopen(req, timeout=60) as r:
        return r.read()

def find_records(root, record_tag="PricesFlexibilityChargeIntraday"):
    """Return all data-record elements (skip the <schema> header)."""
    recs = []
    for el in root.iter():
        name = strip_ns(el.tag)
        if name == "schema":
            continue
        if name == record_tag:
            recs.append(el)
    return recs

def record_to_dict(el):
    """Flatten one record: child elements -> columns; include any attributes too."""
    row = dict(el.attrib)
    for c in el:
        k = strip_ns(c.tag)
        v = (c.text or "").strip()
        # If there are nested children (rare), you can extend this to flatten deeper.
        if k in row and v:
            # avoid collisions if both attr and child share a name
            row[k + "_val"] = v
        else:
            row[k] = v
    return row

def write_csv(rows, path="flex_intraday.csv"):
    # build header as union of keys, but put likely fields first if present
    keys = set().union(*(r.keys() for r in rows)) if rows else set()
    preferred = ["GasDay","From","To","Position","Timestamp","Charge","Quantity","Cost","Currency"]
    fieldnames = [k for k in preferred if k in keys] + [k for k in sorted(keys) if k not in preferred]
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in rows:
            w.writerow(r)
    return path, fieldnames

def main():
    try:
        xml_bytes = fetch_xml(URL)
    except Exception as e:
        print("HTTP error:", e, file=sys.stderr); return

    try:
        root = ET.fromstring(xml_bytes)
    except ET.ParseError as e:
        print("XML parse error:", e, file=sys.stderr)
        print("First 400 bytes:", xml_bytes[:400]); return

    # Grab the actual data elements (not the schema)
    rec_elems = find_records(root, "PricesFlexibilityChargeIntraday")
    if not rec_elems:
        print("No data records found. (Possible reasons: outside gas day, or structure different today.)")
        return

    rows = [record_to_dict(el) for el in rec_elems if list(el) or el.attrib]
    if not rows:
        print("Records were found, but contained neither attributes nor child text.")
        return

    print(f"Found {len(rows)} records. Preview:")
    for i, r in enumerate(rows[:5], 1):
        print(f"{i:>2}. {r}")

    path, cols = write_csv(rows)
    print(f"\nSaved {len(rows)} rows to {path}")
    print("Columns:", cols)

if __name__ == "__main__":
    main()

Found 15 records. Preview:
 1. {'schema': '', 'PricesFlexibilityChargeIntraday': ''}
 2. {'Timestamp': '2025-11-10T06:00:00', 'FlexibilityQuantityPreliminary': '0.000', 'UnitFlexibilityQuantityPreliminary': 'MWh', 'FlexibilityCostsPreliminary': '0.0000', 'UnitFlexibilityCostsPreliminary': 'EUR', 'FlexibilityChargePreliminary': '0.0000', 'UnitFlexibilityChargePreliminary': 'EUR/MWh'}
 3. {'Timestamp': '2025-11-10T07:00:00', 'FlexibilityQuantityPreliminary': '0.000', 'UnitFlexibilityQuantityPreliminary': 'MWh', 'FlexibilityCostsPreliminary': '0.0000', 'UnitFlexibilityCostsPreliminary': 'EUR', 'FlexibilityChargePreliminary': '0.0000', 'UnitFlexibilityChargePreliminary': 'EUR/MWh'}
 4. {'Timestamp': '2025-11-10T08:00:00', 'FlexibilityQuantityPreliminary': '0.000', 'UnitFlexibilityQuantityPreliminary': 'MWh', 'FlexibilityCostsPreliminary': '0.0000', 'UnitFlexibilityCostsPreliminary': 'EUR', 'FlexibilityChargePreliminary': '0.0000', 'UnitFlexibilityChargePreliminary': 'EUR/MWh'}
 5. {'Timest